In [1]:
#  -------------------------------------------
"""
입출력 양식을 
준수해 주십시오

###  INPUT ###
import pandas as pd
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

{
    
        Write codes...
    
    Training model name : model
    ex) y_pred = model.predict(X_test)
    

}


### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)
"""

'\n입출력 양식을 \n준수해 주십시오\n\n###  INPUT ###\nimport pandas as pd\ninput_data = pd.read_csv(\'2023_smartFarm_AI_hackathon_dataset.csv\')\n\n{\n    \n        Write codes...\n    \n    Training model name : model\n    ex) y_pred = model.predict(X_test)\n    \n\n}\n\n\n### OUTPUT ###\nprint("RMSE:", rmse)\nprint("R2_score:", r2score)\n'

In [1]:


# ------------------------------------------------


#      SAMPLE MODEL


# ------------------------------------------------


###  INPUT ###
"""

Read CSV files from the given list of file paths and return DataFrames.

"""

import pandas as pd

input_data = pd.read_csv("2023_smartFarm_AI_hackathon_dataset.csv")
#  -------------------------------------------
###    Write codes...  ###
#      EXAMPLE CODE      #




# Read CSV files from the lists of file paths


# Now you have lists of DataFrames for each type of data 

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping



# EDA
#import matplotlib.pyplot as plt
# import seaborn as sns

# 통계적 특성 확인
print(input_data.describe())

# 상관계수 히트맵
# plt.figure(figsize=(12, 10))
# sns.heatmap(data.corr(), annot=True, cmap='coolwarm')
# plt.show()


# ... (Data preprocessing code here)
# 0을 NaN으로 변경
input_data.replace(0, np.nan, inplace=True)

# 각 열별로 결측치 개수 확인
missing_values = input_data.isnull().sum()
print(missing_values)

input_data = input_data.drop(['frmDist', 'date', 'frmYear', 'lefstalklt', 'daysuplyqy', 'frtstSetCo', 'pllnLt', 'flanJnt', 'hvstJnt', 'flwrCo', 'frtstJnt'], axis=1)  # 날짜와 특정 특성은 제거합니다.

# Split the data into training and testing sets
X = input_data[input_data.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]

Y = input_data[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# Random Forest
multi_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100))
multi_rf.fit(X_train, y_train)
rf_pred = multi_rf.predict(X_test)

# SVM
multi_svm = MultiOutputRegressor(SVR())
multi_svm.fit(X_train, y_train)
svm_pred = multi_svm.predict(X_test)

# LSTM
X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

model = Sequential()
model.add(LSTM(50, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model.add(Dense(2)) # 두 개의 타깃 값을 예측하기 위해 유닛 수를 2로 설정
model.compile(optimizer='adam', loss='mse')

# Early stopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# 모델 학습
model.fit(X_train_lstm, y_train, validation_split=0.2, epochs=50, verbose=1, callbacks=[early_stopping])


lstm_predictions_train = model.predict(X_train_lstm)

X_train_rf = np.hstack((X_train, lstm_predictions_train))

rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train_rf, y_train)

lstm_predictions_test = model.predict(X_test_lstm)
X_test_rf = np.hstack((X_test, lstm_predictions_test))

y_pred = rf.predict(X_test_rf)



# 세 모델의 예측을 평균
#y_pred = (rf_pred + svm_pred + lstm_pred) / 3.0

# # ... (Data preprocessing code here)
# input_data = input_data.drop(columns=['frmDist'])

# # Split the data into training and testing sets
# X = input_data[input_data.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
# Y = input_data[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
# X_train, X_test, y_train, y_test = train_test_split(
#     X, Y, test_size=0.2, random_state=42
# )


# # Initialize and train the LinearRegression model
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Predict 'y' values using the trained model
# y_pred = model.predict(X_test)


# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_test, y_pred)


rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

# ------------------------------------------------
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)


ImportError: Could not find the DLL(s) 'msvcp140_1.dll'. TensorFlow requires that these DLLs be installed in a directory that is named in your %PATH% environment variable. You may install these DLLs by downloading "Microsoft C++ Redistributable for Visual Studio 2015, 2017 and 2019" for your platform from this URL: https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads